In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!pip install -q transformers nltk gensim scikit-learn torch pandas spacy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import re
import nltk
import gensim
import torch
from transformers import BertTokenizer, BertModel
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

# 确保 nltk 相关资源已下载
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
train = pd.read_csv('/content/drive/My Drive/SemEval2022-Task6-Sarcasm-Detection/Data/Train_Dataset.csv')
test = pd.read_csv('/content/drive/My Drive/SemEval2022-Task6-Sarcasm-Detection/Data/Test_Dataset.csv')

# 预处理文本（去掉特殊字符，转换为小写）
def preprocess_text(text):
    return re.sub(r'[^a-zA-Z0-9 ]', '', text.lower())

X_train, y_train = train['tweet'].apply(preprocess_text), train['sarcastic']
X_test, y_test = test['tweet'].apply(preprocess_text), test['sarcastic']


In [6]:
print(X_train)

0       the only thing i got from college is a caffein...
1       i love it when professors draw a big question ...
2       remember the hundred emails from companies whe...
3       today my poppop told me i was not forced to go...
4       volphancarol littlewhitty mysticalmanatee i di...
                              ...                        
6929        the spike in in months is about be ridiculous
6930    think in second to last class of the year my s...
6931    finally surfacing after holiday to scotland di...
6932    couldnt be today done every got their gcses th...
6933    overheard my year old a friend smell like tart...
Name: tweet, Length: 6934, dtype: object


In [7]:
def compute_tfidf_features(X_train, X_test):
    count_vect = CountVectorizer()
    X_counts = count_vect.fit_transform(X_train)

    tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
    X_tfidf = tfidf_transformer.fit_transform(X_counts)

    return X_tfidf[:len(X_train)], X_tfidf[len(X_train):]

X_train_tfidf, X_test_tfidf = compute_tfidf_features(X_train, X_test)


In [8]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def compute_bert_embeddings(texts, model, tokenizer, batch_size=32, max_length=128):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt').to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        embeddings.extend(batch_embeddings)

        # 释放显存
        torch.cuda.empty_cache()

    return np.array(embeddings)

# 计算 BERT 特征
X_train_bert = compute_bert_embeddings(X_train.tolist(), bert_model, tokenizer, batch_size=32)
X_test_bert = compute_bert_embeddings(X_test.tolist(), bert_model, tokenizer, batch_size=32)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
import numpy as np
import spacy
from gensim.models import Word2Vec
from tqdm import tqdm

# 加载 spaCy 英文模型
nlp = spacy.load("en_core_web_sm")

# 使用 spaCy 进行分词，去掉停用词和标点
def spacy_tokenizer(text):
    doc = nlp(text)
    return [token.text.lower() for token in doc if not token.is_stop and not token.is_punct]

# 训练 Word2Vec
print("Tokenizing training data using spaCy...")
sentences = [spacy_tokenizer(text) for text in tqdm(X_train)]

print("Training Word2Vec model...")
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

# 计算 Word2Vec 词向量的均值
def mean_embedding(vectors):
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros((100,))  # 100 维度默认向量

# 计算 Word2Vec 特征
def compute_word2vec_embeddings(texts, model):
    tokenized_texts = [spacy_tokenizer(text) for text in tqdm(texts, desc="Tokenizing")]

    embeddings = []
    for words in tqdm(tokenized_texts, desc="Generating Embeddings"):
        vectors = [model.wv[word] for word in words if word in model.wv.key_to_index]
        embeddings.append(mean_embedding(vectors))

    return np.array(embeddings)

# 计算训练集和测试集的 Word2Vec 特征
print("Computing Word2Vec embeddings for training data...")
X_train_w2v = compute_word2vec_embeddings(X_train.tolist(), word2vec_model)

print("Computing Word2Vec embeddings for testing data...")
X_test_w2v = compute_word2vec_embeddings(X_test.tolist(), word2vec_model)


Tokenizing training data using spaCy...


100%|██████████| 6934/6934 [01:09<00:00, 99.92it/s] 


Training Word2Vec model...
Computing Word2Vec embeddings for training data...


Generating Embeddings: 100%|██████████| 6934/6934 [00:00<00:00, 36198.25it/s]


Computing Word2Vec embeddings for testing data...


Generating Embeddings: 100%|██████████| 1400/1400 [00:00<00:00, 38101.58it/s]


In [12]:
import time
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

class_weight = {1: 3, 0: 1}

# 🚀 训练 SVM 的通用函数
def train_svm(X_train, X_test, y_train, y_test, label):
    print(f"🚀 Training {label} SVM ...")

    start_time = time.time()  # ⏳ 记录开始时间

    clf = SVC(C=10, kernel='rbf', class_weight=class_weight)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    end_time = time.time()  # ⏳ 记录结束时间
    elapsed_time = end_time - start_time  # 计算总耗时

    print(f"{label} ✅ Done in {elapsed_time:.2f} seconds!")
    print(f"{label} Accuracy: {acc:.4f}")
    print(f"{label} F1-score: {f1:.4f}\n")

# 🎯 运行 SVM 训练（单独执行每个任务）
def train_tfidf_svm():
    train_svm(X_train_tfidf.toarray(), X_test_tfidf.toarray(), y_train, y_test, label="TF-IDF SVM")

def train_bert_svm():
    train_svm(X_train_bert, X_test_bert, y_train, y_test, label="BERT SVM")

def train_word2vec_svm():
    train_svm(X_train_w2v, X_test_w2v, y_train, y_test, label="Word2Vec SVM")


In [13]:
train_tfidf_svm()

🚀 Training TF-IDF SVM SVM ...


ValueError: Found array with 0 sample(s) (shape=(0, 10904)) while a minimum of 1 is required by SVC.

In [14]:
train_bert_svm()


🚀 Training BERT SVM SVM ...
BERT SVM ✅ Done in 27.38 seconds!
BERT SVM Accuracy: 0.7264
BERT SVM F1-score: 0.3385



In [15]:
train_word2vec_svm()


🚀 Training Word2Vec SVM SVM ...
Word2Vec SVM ✅ Done in 4.57 seconds!
Word2Vec SVM Accuracy: 0.5143
Word2Vec SVM F1-score: 0.2444



In [16]:
print("X_train_tfidf shape:", X_train_tfidf.shape)
print("X_test_tfidf shape:", X_test_tfidf.shape)


X_train_tfidf shape: (6934, 10904)
X_test_tfidf shape: (0, 10904)


In [17]:
print("X_test length:", len(X_test))
print("First few samples in X_test:", X_test[:5])


X_test length: 1400
First few samples in X_test: 0    size on the the toulouse team that pack is mon...
1                                              pinball
2    so the scottish government want people to get ...
3    villainous pro tip  change the device name on ...
4                       i would date any of these men 
Name: tweet, dtype: object


In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

def compute_tfidf_features(X_train, X_test):
    count_vect = CountVectorizer()
    X_counts_train = count_vect.fit_transform(X_train)  # 训练集: 计算词频矩阵
    X_counts_test = count_vect.transform(X_test)  # 测试集: 直接转换

    tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
    X_tfidf_train = tfidf_transformer.fit_transform(X_counts_train)  # 训练集: 计算 TF-IDF
    X_tfidf_test = tfidf_transformer.transform(X_counts_test)  # 测试集: 直接转换

    return X_tfidf_train, X_tfidf_test  # 分开返回训练和测试的 TF-IDF

X_train_tfidf, X_test_tfidf = compute_tfidf_features(X_train, X_test)


In [20]:
train_tfidf_svm()

🚀 Training TF-IDF SVM SVM ...
TF-IDF SVM ✅ Done in 534.81 seconds!
TF-IDF SVM Accuracy: 0.8407
TF-IDF SVM F1-score: 0.1255



In [21]:
import joblib

# 训练 SVM 并保存模型
def train_svm_and_save(X_train, X_test, y_train, y_test, label, model_path):
    print(f"🚀 Training {label} SVM ...")

    start_time = time.time()  # 记录开始时间
    clf = SVC(C=10, kernel='rbf', class_weight=class_weight)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    end_time = time.time()  # 记录结束时间
    elapsed_time = end_time - start_time  # 计算总耗时

    print(f"{label} ✅ Done in {elapsed_time:.2f} seconds!")
    print(f"{label} Accuracy: {acc:.4f}")
    print(f"{label} F1-score: {f1:.4f}\n")

    # 保存模型
    joblib.dump(clf, model_path)
    print(f"✅ {label} SVM model saved at {model_path}")

# 运行 SVM 并保存模型
train_svm_and_save(X_train_tfidf.toarray(), X_test_tfidf.toarray(), y_train, y_test, "TF-IDF SVM", "tfidf_svm_model.pkl")
train_svm_and_save(X_train_bert, X_test_bert, y_train, y_test, "BERT SVM", "bert_svm_model.pkl")
train_svm_and_save(X_train_w2v, X_test_w2v, y_train, y_test, "Word2Vec SVM", "word2vec_svm_model.pkl")


🚀 Training TF-IDF SVM SVM ...
TF-IDF SVM ✅ Done in 549.20 seconds!
TF-IDF SVM Accuracy: 0.8407
TF-IDF SVM F1-score: 0.1255

✅ TF-IDF SVM SVM model saved at tfidf_svm_model.pkl
🚀 Training BERT SVM SVM ...
BERT SVM ✅ Done in 33.03 seconds!
BERT SVM Accuracy: 0.7264
BERT SVM F1-score: 0.3385

✅ BERT SVM SVM model saved at bert_svm_model.pkl
🚀 Training Word2Vec SVM SVM ...
Word2Vec SVM ✅ Done in 5.89 seconds!
Word2Vec SVM Accuracy: 0.5143
Word2Vec SVM F1-score: 0.2444

✅ Word2Vec SVM SVM model saved at word2vec_svm_model.pkl


In [22]:
# 加载模型
tfidf_svm = joblib.load("tfidf_svm_model.pkl")
bert_svm = joblib.load("bert_svm_model.pkl")
w2v_svm = joblib.load("word2vec_svm_model.pkl")

# 进行预测并打印测试集结果
y_pred_tfidf = tfidf_svm.predict(X_test_tfidf.toarray())
y_pred_bert = bert_svm.predict(X_test_bert)
y_pred_w2v = w2v_svm.predict(X_test_w2v)

print("📊 **Evaluation Metrics**")
print(f"TF-IDF SVM Accuracy: {accuracy_score(y_test, y_pred_tfidf):.4f}, F1-score: {f1_score(y_test, y_pred_tfidf):.4f}")
print(f"BERT SVM Accuracy: {accuracy_score(y_test, y_pred_bert):.4f}, F1-score: {f1_score(y_test, y_pred_bert):.4f}")
print(f"Word2Vec SVM Accuracy: {accuracy_score(y_test, y_pred_w2v):.4f}, F1-score: {f1_score(y_test, y_pred_w2v):.4f}")


📊 **Evaluation Metrics**
TF-IDF SVM Accuracy: 0.8407, F1-score: 0.1255
BERT SVM Accuracy: 0.7264, F1-score: 0.3385
Word2Vec SVM Accuracy: 0.5143, F1-score: 0.2444


In [23]:
from google.colab import files
files.download("tfidf_svm_model.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
from google.colab import files
files.download("bert_svm_model.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
from google.colab import files
files.download("word2vec_svm_model.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>